MusicLM - using LucidRain's pytorch implementation. 

In [ ]:
!pip install musiclm-pytorch

In [ ]:
# !pip install audiolm-pytorch this mgiht be needed idk

In [ ]:
!pip install encodec

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import torch
from musiclm_pytorch import MuLaN, AudioSpectrogramTransformer, TextTransformer
from encodec import EncodecModel
from encodec.utils import convert_audio
from PIL import Image
import numpy as np
import torchaudio
import torch
from IPython.display import Audio
import torchvision.transforms as transforms  
import matplotlib.pyplot as plt

In [ ]:
audio_transformer = AudioSpectrogramTransformer(
    dim = 512,
    depth = 6,
    heads = 8,
    dim_head = 64,
    spec_n_fft = 128,
    spec_win_length = 24,
    spec_aug_stretch_factor = 0.8
)

text_transformer = TextTransformer(
    dim = 512,
    depth = 6,
    heads = 8,
    dim_head = 64
)

mulan = MuLaN(
    audio_transformer = audio_transformer,
    text_transformer = text_transformer
)

# get a ton of <sound, text> pairs and train

wavs = torch.randn(2, 1024)
texts = torch.randint(0, 20000, (2, 256))

loss = mulan(wavs, texts)
loss.backward()

# after much training, you can embed sounds and text into a joint embedding space
# for conditioning the audio LM

In [ ]:
embeds = mulan.get_audio_latents(wavs)  # during training ?? what does this mean, during training vs inference?

embeds = mulan.get_text_latents(texts)  # during inference, I mean do these bits even need to run, this embeds would overwrite the one above

#To do data loader. 

But what does this mean? 

In [ ]:
ds_path = "/drive/blah/audiodataset"

In [3]:
wavs = torch.randn(2, 1024)
texts = torch.randint(0, 20000, (2, 256))

In [4]:
wavs.shape, texts.shape

(torch.Size([2, 1024]), torch.Size([2, 256]))

With Encodec:

This bit can go, just the demo of from the Encodec repo:

In [ ]:
# Instantiate a pretrained EnCodec model
encodec = EncodecModel.encodec_model_24khz()
# The number of codebooks used will be determined bythe bandwidth selected.
# For a bandwidth of 6kbps, `n_q = 8` codebooks are used.
# Supported bandwidths are 1.5kbps (n_q = 2), 3 kbps (n_q = 4), 6 kbps (n_q = 8) and 12 kbps (n_q =16) and 24kbps (n_q=32).
encodec.set_target_bandwidth(6) # 6 kbps to match the sound stream n_q of 8

# Load and pre-process the audio waveform
wav, sr = torchaudio.load("/content/drive/MyDrive/16bitDrums/Beat162.wav")
nwav = wav[0][:].unsqueeze(0)

# print(nwav.shape)
wav = convert_audio(nwav, sr, encodec.sample_rate, encodec.channels)
wav = wav.unsqueeze(0)

# Extract discrete codes from EnCodec
with torch.no_grad():
    encoded_frames = encodec.encode(wav)
codes = torch.cat([encoded[0] for encoded in encoded_frames], dim=-1)  # [B, n_q, T]

# Remember to chunk the code book's t to 1024

In [ ]:
from musiclm_pytorch import MuLaNEmbedQuantizer
# setup the quantizer with the namespaced conditioning embeddings, unique per quantizer as well as namespace (per transformer)
quantizer = MuLaNEmbedQuantizer(
    mulan = mulan,                          # pass in trained mulan from above
    conditioning_dims = (1024, 1024, 1024), # say all three transformers have model dimensions of 1024
    namespaces = ('semantic', 'coarse', 'fine')
)
# now say you want the conditioning embeddings for semantic transformer
wavs = torch.randn(2, 1024)
conds = quantizer(wavs = wavs, namespace = 'semantic') # (2, 8, 1024) - 8 is number of quantizers, dunno what the two is about, stereo audio?

#Semantic Transformer:

In [ ]:
import torch
from audiolm_pytorch import HubertWithKmeans, SemanticTransformer, SemanticTransformerTrainer

# work out if we need to be getting or dl the checkpoints if not already there
wav2vec = HubertWithKmeans(
    checkpoint_path = './hubert/hubert_base_ls960.pt',
    kmeans_path = './hubert/hubert_base_ls960_L9_km500.bin'
)

semantic_transformer = SemanticTransformer(
    num_semantic_tokens = wav2vec.codebook_size,
    dim = 1024,
    depth = 6,
    audio_text_condition = True      # this must be set to True (same for CoarseTransformer and FineTransformers)
).cuda()

trainer = SemanticTransformerTrainer(
    transformer = semantic_transformer,
    wav2vec = wav2vec,
    audio_conditioner = quantizer,   # pass in the MulanEmbedQuantizer instance above
    folder = ds_path,
    batch_size = 1,
    data_max_length = 320 * 32,
    num_train_steps = 1000 # idk the number of training steps needed here for the semantic transformer
)

trainer.train()

# Coarse Transformer:

In [ ]:
from audiolm_pytorch import CoarseTransformer, CoarseTransformerWrapper, CoarseTransformerTrainer, FineTransformer, FineTransformerWrapper, FineTransformerTrainer, AudioLM

In [ ]:
wav2vec = HubertWithKmeans(
    checkpoint_path = f'./{hubert_ckpt}',
    kmeans_path = f'./{hubert_quantizer}'
)

# Instantiate a pretrained EnCodec model
encodec = EncodecModel.encodec_model_24khz()
encodec.set_target_bandwidth(6) # 6 kbps to match the sound stream n_q of 8

coarse_transformer = CoarseTransformer(
    num_semantic_tokens = wav2vec.codebook_size,
    codebook_size = 1024,
    num_coarse_quantizers = 3,
    dim = 512,
    depth = 6,
    audio_text_condition = True # adding audio text cond true    
)

trainer = CoarseTransformerTrainer(
    transformer = coarse_transformer,
    codec = encodec, # change to encodec
    wav2vec = wav2vec,
    folder = ds_path,
    batch_size = 1,
    data_max_length = 320 * 32,
    save_results_every = 1000,
    save_model_every = 1000,
    num_train_steps = 10000,
    
)
# NOTE: I changed num_train_steps to 9 (aka 8 + 1) from 10000 to make things go faster for demo purposes
# adjusting save_*_every variables for the same reason

trainer.train()

#Fine transformer

In [ ]:
encodec = EncodecModel.encodec_model_24khz()
encodec.set_target_bandwidth(6) # 6 kbps to match the sound stream n_q of 8

fine_transformer = FineTransformer(
    num_coarse_quantizers = 3, # could test 2 by 2 if using n_q = 4, or 1 by 3
    num_fine_quantizers = 5,
    codebook_size = 1024,
    dim = 512,
    depth = 6,
    audio_text_condition = True   # adding audio text cond true
)

trainer = FineTransformerTrainer(
    transformer = fine_transformer,
    codec = encodec, # change to encodec
    folder = ds_path,
    batch_size = 1,
    data_max_length = 320 * 32,
    num_train_steps = 10000,
    save_model_every = 1000
)
# NOTE: I changed num_train_steps to 9 (aka 8 + 1) from 10000 to make things go faster for demo purposes
# adjusting save_*_every variables for the same reason
trainer.train()

#Inference:

In [ ]:
# Everything together
audiolm = AudioLM(
    wav2vec = wav2vec,
    codec = encodec,
    semantic_transformer = semantic_transformer,
    coarse_transformer = coarse_transformer,
    fine_transformer = fine_transformer
)

generated_wav = audiolm(batch_size = 1)

In [ ]:
# you need the trained AudioLM (audio_lm) from above
# with the MulanEmbedQuantizer (mulan_embed_quantizer)
from musiclm_pytorch import MusicLM

musiclm = MusicLM(
    audio_lm = audio_lm, # ? do we need (batch_size = 1) here as well?
    mulan_embed_quantizer = mulan_embed_quantizer
)

music = musiclm('the crystalline sounds of the piano in a ballroom', num_samples = 4) # sample 4 and pick the top match with mulan